In [1]:
'https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/'

'https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/'

In [1]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [2]:
main_df = pd.read_csv('C:/Users/user/Desktop/mlmd/sentiment_tweets3.csv')
print(main_df.shape)
main_df = main_df[["message", "label"]]


(10314, 3)


In [12]:
df = pd.DataFrame({'label':main_df.label, 'text':main_df.message})

In [13]:
df['label'].value_counts()

0    8000
1    2314
Name: label, dtype: int64

In [14]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [15]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [17]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.2, random_state = 1234)

In [18]:
df_trn.shape, df_val.shape

((8251, 2), (2063, 2))

In [19]:
df_trn.head()

,label,text
6744,0,RogerInTDot Don let motivation Staying fit hea...
241,0,Lizinhollywood Can wait
8472,1,Bayern missing chances Real winningReal winnin...
5777,0,jbfanforever Thank sooo much though getting info
1250,0,essay cultural context Inside I Dancing done s...


In [20]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [22]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)

In [23]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.404914,5.331501,0.187404,00:03


In [24]:
learn.save_encoder('ft_enc')

In [31]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.7)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (8251 items)
x: TextList
xxbos xxunk xxmaj don let motivation xxmaj staying fit healthy motivation enough,xxbos xxmaj xxunk xxmaj can wait,xxbos xxmaj bayern missing chances xxmaj real xxunk winning xxmaj ronaldo fanboys come droves xxmaj ronaldo fanboys come droves xxmaj depression xxmaj therefore xxmaj bayern missing chances xxmaj depression,xxbos xxunk xxmaj thank sooo much though getting info,xxbos essay cultural context xxmaj inside i xxmaj dancing done shower food
y: CategoryList
0,0,1,0,0
Path: .;

Valid: LabelList (2063 items)
x: TextList
xxbos home school got dance class soon,xxbos xxunk mmh sounds really yummy,xxbos xxmaj xxunk xxmaj and good morning,xxbos xxunk i gon na check online tmrw,xxbos xxup xxunk xxmaj yes nobody going spend bn year digging filling xxunk lol xxmaj this kind economic tool used depression worked well
y: CategoryList
0,0,0,0,1
Path: .;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (m

In [32]:
learn.fit_one_cycle(20, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.609595,0.413124,0.883665,01:19
1,0.527059,0.406233,0.882210,01:20
2,0.469001,0.274082,0.925351,01:20
3,0.418181,0.237119,0.920989,01:20
4,0.389466,0.230435,0.910325,01:19
5,0.380880,0.220788,0.919535,01:20
6,0.359701,0.212304,0.914687,01:19
7,0.381986,0.213349,0.928260,01:20
8,0.354453,0.209482,0.936015,01:20
9,0.395371,0.211834,0.927775,01:20


In [33]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,1573,109
1,27,354
